In [ ]:
from flask import Flask, render_template, request
import sys
import cx_Oracle
import os
import pandas as pd
    
app = Flask(__name__)

@app.route("/curation",  methods=['GET'])
def index(): 
            usercode = request.args.get('usercode')
            res = curation(usercode)        
            return res


    
def curation(usercode):
    LOCATION = r"C:\oraclexe"
    os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
    
    dsn=cx_Oracle.makedsn('localhost',1521,'xe')
    conn=cx_Oracle.connect('hr','a1234')

    cursor=conn.cursor()
    
    df=pd.read_sql('select usercode, moviecode, count(*) as count from ugotfilm_movie_choice group by usercode, moviecode order by usercode', con=conn)
    
    user= int(usercode)
    
    recommendation_pivot = df.pivot(index='USERCODE', columns='MOVIECODE', values='COUNT')
    recommendation_pivot.fillna(0, inplace=True)
    
    def movie_seen(user_id):
      return recommendation_pivot.loc[user_id][recommendation_pivot.loc[user_id]>0]

    def nearest_user(user_id, n):
      return small_test_corr.loc[user_id].sort_values(ascending=False)[1:n+1]

    small_test = recommendation_pivot.T
    small_test_corr = small_test.corr()
    
    small_test_corr[user].sort_values(ascending=False)[1:11]
    res = nearest_user(user,1)
    
    unseen = set(movie_seen(res.index[0]).index) - set(movie_seen(user).index)
    return str(unseen) 

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
